# **Extract data through Google API**

In [80]:
import json
from typing import List, Dict, Any, Tuple
import pandas as pd
import geopandas as gpd
import requests
import datetime
import matplotlib.pyplot as plt

##### **Steps**:
To get the data from the Google Bucket into workable format, we follow:
* Connect with the API
* **Expand this step!** Set-up pipeline which downloads all relevant data per country code
* Format the data into DateTime (y) and features (x)
* Store externally as .csv

##### **Background info -- API**
The API has three basic functions:
* *ListGauges*: Returns a list of supported gauges within a given country (code)
* *GetGaugeModel*: Returns model information of a specific gauge (within a given country, that is)
* *QueryGaugeForecasts*: Returns the forecasts of specific gauge

In [81]:
def get_API_key(path : str) -> str:
    """
    Get the API key from a .txt file stored elsewhere

    :param path: path to the .txt file containing the API key
    :return: the API key
    """
    with open(path, 'r') as f:
        return f.read().strip()

# Note: add this folder manually at the pointed path for the script to run
# For the key, ask the owner of the repository or try brute force :)
path_API_key = '../../data/keys/API_key.txt'

In [82]:
def get_country_codes(path : str) -> Dict:
    """
    Get the country codes from a .json file stored elsewhere

    :param path: path to the .json file containing the country codes
    :return: the country codes
    """
    with open(path, 'r') as f:
        return json.load(f)


path_country_codes = '../../data/country_codes.json'
dict_country_codes = get_country_codes(path_country_codes)

In [83]:
def generate_url_ListGauges(path_to_key : str) -> str:
    """
    Generate the URL to get the list of gauges for a specific country

    :param path_to_key: path to the .txt file containing the API key
    :return: the URL
    """
    base_url = 'https://floodforecasting.googleapis.com/v1/gauges:searchGaugesByArea'
    return f'{base_url}?key={get_API_key(path_to_key)}'

In [91]:
def make_request_ListGauges(country_code : str, path_to_key : str) -> List[Dict[str, Any]]:
    """
    Make the API request for the list of gauges for a specific country

    :param url: the URL
    :return: the list of gauges
    """
    response = requests.post(
        generate_url_ListGauges(path_to_key),
        json = {'regionCode': country_code}
    )

    if response.status_code != 200:
        raise Exception(f'Error: {response.status_code} -- {response.text}')
    
    return response


def get_ListGauges(country_code : str, path_to_key : str) -> List[Dict[str, Any]]:
    """
    Get the list of gauges for a specific country, and verify the response

    :param country_code: the country code
    :param path_to_key: path to the .txt file containing the API key
    :return: the list of gauges
    """
    response = make_request_ListGauges(country_code, path_to_key)
    try:
        return response.json()['gauges']
    except ValueError as exc:
        raise Exception(f'Error parsing .json: {exc} -- {response.text}')

In [104]:
def print_gauges(gauges : List[Dict[str, Any]]) -> None:
    """
    Print the list of gauges

    :param gauges: the list of gauges
    """
    for gauge in gauges:
        print(f"{gauge['siteName']} ({gauge['gaugeId']})")

In [97]:
list_gauges = get_ListGauges('ML', path_API_key)

In [100]:
print(type(list_gauges))
print(len(list_gauges))
print(list_gauges)

<class 'list'>
16
[{'location': {'latitude': 11.22291666666553, 'longitude': -8.485416666669321}, 'siteName': '', 'source': 'HYBAS', 'river': 'Sankarani', 'gaugeId': 'hybas_1120766460', 'qualityVerified': True}, {'location': {'latitude': 11.427083333332233, 'longitude': -6.581250000002682}, 'siteName': '', 'source': 'HYBAS', 'river': '', 'gaugeId': 'hybas_1120758950', 'qualityVerified': True}, {'location': {'latitude': 12.018749999998988, 'longitude': -8.322916666669414}, 'siteName': '', 'source': 'HYBAS', 'river': '', 'gaugeId': 'hybas_1120737100', 'qualityVerified': True}, {'location': {'latitude': 13.493749999998954, 'longitude': -6.202083333336134}, 'siteName': '', 'source': 'HYBAS', 'river': '', 'gaugeId': 'hybas_1120679780', 'qualityVerified': True}, {'location': {'latitude': 12.410416666665697, 'longitude': -8.106250000002774}, 'siteName': '', 'source': 'HYBAS', 'river': 'Niger', 'gaugeId': 'hybas_1121919510', 'qualityVerified': True}, {'location': {'latitude': 13.36041666666562